In [1]:
import numpy as np
import time
from fastai.vision.all import *
from ipywidgets import Label, Button, FileUpload, Output, VBox, AppLayout, Layout, Dropdown
import urllib
import requests
from bs4 import BeautifulSoup
from IPython.core.display import HTML
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

# Get inference learner
learn_inf = load_learner('export.pkl')

def getCharactersImageFromWiki(simpsons_character):
    wiki = f'https://simpsonswiki.com/wiki/{simpsons_character}'
    try:
        r = urllib.request.urlopen(wiki).read()
        soup = BeautifulSoup(r, 'html.parser')
        images = soup.findAll('img')
        urls = [image['src'] for image in images if simpsons_character in image['src']]
        img = PILImage.create(requests.get(urls[0], stream=True).raw)
        succeeded = True
    except:
        succeeded = False
        img = None
    
    return succeeded, img

/srv/conda/envs/notebook/lib/python3.7/site-packages/fastcore/foundation.py:51: UserWarning: `patch_property` is deprecated and will be removed; use `patch(as_prop=True)` instead
  warnings.warn("`patch_property` is deprecated and will be removed; use `patch(as_prop=True)` instead")


In [2]:
%%html
<style>
@import url('https://fonts.googleapis.com/css2?family=Gochi+Hand&display=swap');
body {
  font-family: 'Gochi Hand', cursive;
}
.out_style{
    color: black;
    background-color:yellow;
    font-family: 'Gochi Hand', cursive;
}
.gochihand {
    font-family: 'Gochi Hand', cursive;
}
</style>

In [3]:
VERBOSE = False

TEXTS = {"btn_header" : {"en" : "Simpson Character Detector", "de" : "Simpsons Charakter Detektor"},
         "dpd_lang" : {"en" : "Language", "de" : "Sprache"},
         "btn_doc" : {"en" : "Show Info", "de" : "Info Anzeigen"},
         "btn_upload" : {"en" : "Upload Image", "de" : "Bild hochladen"},
         "btn_status_init" : {"en" : "", "de" : ""},
         "btn_status_progress" : {"en" : "Please Wait... - ", "de" : "Bitte Warten... - "},
         "btn_status_default" : {"en" : "Please Wait", "de" : "Bitte Warten"},
         "btn_status_load" : {"en" : "Loading Image", "de" : "Bild wird geladen"},
         "btn_status_detect" : {"en" : "Detecting Simpson Character", "de" : "Simpson Charakter werden erkannt"},
         "btn_status_ready" : {"en" : "Ready For User Input", "de" : "Bereit für Benutzereingabe"},
         "btn_status_error" : {"en" : "Error Loading Image", "de" : "Fehler beim Abruf des Bildes"},
         "select_image" : {"en" : "Please Select An Valid Image File", "de" : "Bitte wählen Sie eine gültige Bilddatei"},
         "prob" : {"en" : "Probability", "de" : "Wahrscheinlichkeit"}
        }

lang="en"

HTML_EN = """<div class="jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput " data-mime-type="text/markdown">
<h1 id="Simpsons-detector">Simpsons detector<a class="anchor-link" href="#Simpsons-detector">¶</a></h1><p>Upload a picture and find out which Simpsons character is visible, or at least which Simpson's character the picture resembles. After a few seconds a name and a probability will appear.</p>
<p>The predictions are made by an artificial intelligence model. It only knows characters from the comic series Simpsons. No matter which picture is uploaded, the model will always give the name of a Simpsons character.</p>
<p>This is a small project as result of chapter 2 of the book <a href="https://www.amazon.de/Deep-Learning-Coders-Fastai-Pytorch/dp/1492045527">Deep Learning for Coders with fastai and PyTorch: AI Applications Without a PhD</a>. The program code is based on the code in the book and the examples from <a href="http://docs.fast.ai/">fastai</a>.</p>
<p>The Simpsons images are taken from <a href="https://simpsonswiki.com/">https://simpsonswiki.com/</a>.</p>
</div>"""

HTML_DE = """<div class="jp-RenderedHTMLCommon jp-RenderedMarkdown jp-MarkdownOutput " data-mime-type="text/markdown">
<h1 id="Simpsons-Detektor">Simpsons Detektor<a class="anchor-link" href="#Simpsons-Detektor">¶</a></h1><p>Lade ein Bild hoch und finde heraus, welcher Simpsons Charakter zu sehen ist, oder zumindest welchem Simpsons Charakter das Bild ähnelt. Nach einigen Sekunden erscheint ein Name und eine Wahrscheinlichkeit.</p>
<p>Die Vorhersagen werden von einem Künstliche Intelligenz Modell gemacht. Es kennt als Ausgabe nur Figuren aus der Comic-Serie Simpsons. Egal welches Bild hochgeladen wird, das Modell wird immer den Namen eines Simpsons Charakters nennen.</p>
<p>Dies ist ein kleines Projekt als Resultat von Kapitel 2 des Buches <a href="https://www.amazon.de/Deep-Learning-Coders-Fastai-Pytorch/dp/1492045527">Deep Learning for Coders with fastai and PyTorch: AI Applications Without a PhD</a>. Der Programmcode basiert auf dem Code im Buch und den Beispielen aus<a href="http://docs.fast.ai/">fastai</a>.</p>
<p>Die Simpsons-Bilder sind dem <a href="https://simpsonswiki.com/">https://simpsonswiki.com/</a> entnommen.</p>
</div>"""

DOC = {"en" : HTML_EN,
       "de" : HTML_DE}

# defining widgets
dpd_lang = Dropdown(options=['en', 'de'], value='en', 
                    description=TEXTS["dpd_lang"][lang], layout=Layout(height='auto', width='auto'))
btn_doc = Button(description=TEXTS["btn_doc"][lang], layout=Layout(height='auto', width='auto'))
btn_upload = FileUpload(description=TEXTS["btn_upload"][lang], multiple=False, layout=Layout(height='auto', width='auto'))
btn_header = Button(description=TEXTS["btn_header"][lang], disabled=True, layout=Layout(height='auto', width='auto'))
btn_status = Button(description=TEXTS["btn_status_init"][lang], disabled=True, layout=Layout(height='auto', width='auto'))
input_img = Output(clear_output=True)
output = Output(clear_output=True)

# styling
for btn in [btn_header, output]:
    btn.add_class('out_style')
for btn in [btn_status, btn_doc, btn_upload, dpd_lang]:
    btn.add_class('gochihand')


def classify(image):
    name = ''
    if len(btn_upload.value) > 0:
        pred,pred_idx,probs = learn_inf.predict(image)
        name = '_'.join([s.capitalize() for s in pred.split('_')])
        name_space = ' '.join(name.split('_'))
        message = f'<a href="https://simpsonswiki.com/wiki/{name}"">{name_space}</a> {TEXTS["prob"][lang]}: {probs[pred_idx]:.2%}'
    else:
        message = TEXTS["select_image"][lang]
    
    return name, message

# defining event functions
def displayWaitMessage(message=TEXTS["btn_status_default"][lang]):
    btn_status.description = f'{TEXTS["btn_status_progress"][lang]} {message}'
    btn_status.style.button_color = 'orange'

def displayReadyness():
    btn_status.description = TEXTS["btn_status_ready"][lang]
    btn_status.style.button_color = 'lightgreen'

def outputImage(img):
    with input_img:
        output.clear_output()
        displayWaitMessage(TEXTS["btn_status_load"][lang])
        start = time.time()
        if VERBOSE: print(img.size)
        display(img.to_thumb(200))
        displayWaitMessage(TEXTS["btn_status_detect"][lang])
        end = time.time()
        if VERBOSE: print('took', end-start, 'for displaying the image')
        start = time.time()
    with output:
        name, result = classify(img)
        if len(name) > 0:
            ok, char_img = getCharactersImageFromWiki(name)
            if ok:
                display(char_img.to_thumb(200))
        display(HTML(result))
        end = time.time()
        if VERBOSE: print('took', end-start, 'for displaying the preds')
        displayReadyness()


def on_btn_doc_clicked(b):
    output.clear_output()
    input_img.clear_output()
    with output:
        display(HTML(DOC[lang]))
        
def on_data_change(change):
    output.clear_output()
    input_img.clear_output()
    start = time.time()
    img = PILImage.create(BytesIO(btn_upload.value[-1].content))
    end = time.time()
    if VERBOSE: print('took', end-start, 'for loading the image')
    outputImage(img)
    btn_upload._counter = 0
    
def on_lang_select(change):
    global lang 
    lang = dpd_lang.value
    dpd_lang.description=TEXTS["dpd_lang"][lang]
    btn_upload.description=TEXTS["btn_upload"][lang]
    btn_header.description=TEXTS["btn_header"][lang]
    btn_status.description=TEXTS["btn_status_init"][lang]
    btn_doc.description=TEXTS["btn_doc"][lang]

# adding events
btn_doc.on_click(on_btn_doc_clicked)
btn_upload.observe(on_data_change, names=['value'])
dpd_lang.observe(on_lang_select)

# Layout and Style 
applayout1 = AppLayout(left_sidebar=btn_doc,
                       right_sidebar=dpd_lang)
applayout2 = AppLayout(header=btn_status,
                       left_sidebar=input_img,
                       right_sidebar=output,
                       footer=btn_upload)

displayReadyness()

display(VBox([btn_header, applayout1, applayout2]))